<a href="https://colab.research.google.com/github/mking2014/final-4449-king/blob/main/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import datetime

# Load Data

Gender: Male = 0, Female = 1


In [2]:
df = pd.read_csv('/content/age_gender.csv')
df = df.drop(columns = ['img_name'])
print(f"Dataset size: {df.shape}")
df.head()

FileNotFoundError: ignored

In [ ]:
# Converting pixels into numpy array
df['pixels']=df['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

# EDA

In [ ]:
sns.countplot(x="gender", palette='spring', data=df.replace({"gender" : {1 : "male", 0 : "female"}}))
plt.title('Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')

In [ ]:
sns.countplot(x="ethnicity", palette='spring',
              data=df.replace({"ethnicity" : {0: "Caucasian", 1: "Black", 2: "Asian", 3:"Indian", 4:"Hispanic"}}))

plt.title('Ethnicity Distribution')
plt.xlabel('Ethnicity')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize=(12, 4))

sns.histplot(data = df, x='age', color = '#5e8d5a', edgecolor='#ee6f68', kde=True)
plt.title('Age Distribution')
plt.xlabel('Age')

In [ ]:
ethnicity_list = ["White", "Black", "Asian", "Indian", "Hispanic"]
gender_list = ['Male', 'Female']

plt.figure(figsize=(16,16))

for i,a in zip(np.random.randint(0, 23705, 25),range(1,26)):
    plt.subplot(5,5,a)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(df['pixels'].iloc[i].reshape(48, 48), cmap ='gray')
    plt.xlabel(
        "     Age:"+str(df['age'].iloc[i])+
        "  Ethnicity: "+ str(ethnicity_list[df['ethnicity'].iloc[i]])+
        "  Gender:"+ str(gender_list[df['gender'].iloc[i]])
    )

plt.tight_layout()
plt.show()

# Data Preparation

This model will predict the age & gender usge the images. A classification model will be used to predict the gender (binary result) while a regression model will be used to predict the age.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.head()

In [ ]:
X = df['pixels']
# normalizing pixels data
X = df['pixels']/255

# reshape images
X = np.array(df['pixels'].tolist())
X = X.reshape(-1,48,48, 1)
print(f"Shape of images: {X.shape}")

y_age = df['age']
y_gender = df['gender']
print(f"Age array shape: {(y_age.shape)}")
print(f"Gender array shape: {(y_gender.shape)}")

In [ ]:
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(X, y_age, test_size=0.2)
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(X, y_gender, test_size=0.2)

In [ ]:
print(X_train_age.shape)
print(X_test_age.shape)
print(X_train_gender.shape)
print(X_test_gender.shape)

# Model Developmnet

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
%load_ext tensorboard

In [ ]:
input_shape = (48, 48, 1)

age_model = Sequential()
age_model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
age_model.add(MaxPooling2D((2,2)))
age_model.add(Conv2D(64, (3,3), activation='relu'))
age_model.add(MaxPooling2D((2,2)))
age_model.add(Conv2D(128, (3,3), activation='relu'))
age_model.add(MaxPooling2D((2,2)))
age_model.add(Flatten())
age_model.add(Dense(64, activation='relu'))
age_model.add(Dropout(0.5))
age_model.add(Dense(1, activation='relu'))


age_model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['mse', 'mae'])


In [ ]:
age_model.summary()

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

age_model.fit(X_train_age, y_train_age,
              validation_data = (X_test_age, y_test_age),
              epochs = 30,
              validation_split=0.3,
              callbacks = [tensorboard_callback])

In [ ]:
%tensorboard — logdir logs

In [ ]:
# g_model = Sequential()
# g_model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
# g_model.add(MaxPooling2D((2,2)))
# g_model.add(Conv2D(64, (3,3), activation='relu'))
# g_model.add(MaxPooling2D((2,2)))
# g_model.add(Conv2D(128, (3,3), activation='relu'))
# g_model.add(MaxPooling2D((2,2)))
# g_model.add(Flatten())
# g_model.add(Dense(64, activation='relu'))
# g_model.add(Dropout(0.5))
# g_model.add(Dense(1, activation='sigmoid'))

# g_model.compile(loss='binary_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])